# Users analysis

In [0]:
import boto3

ACCESS_KEY_ID = '.'
SECRET_ACCESS_KEY = '.+.'
BUCKET_NAME = 'jedha-cloud-storage-lucy'
PREFIX ="youtube-"
SCHEME = 'S3'
S3_RESOURCE = 's3'
session = boto3.Session(
    aws_access_key_id=ACCESS_KEY_ID,
    aws_secret_access_key=SECRET_ACCESS_KEY
)
s3 = session.resource("s3")
bucket = s3.Bucket(BUCKET_NAME)

# We define a helper function to easily get the full S3 path of an object, given its key
def get_s3_path(key, bucket_name=BUCKET_NAME, scheme=SCHEME):
    return f"{scheme}://{bucket_name}/{key}"

In [0]:
# TODO: Load the file hosted at `filepath` onto a PySpark DataFrame: user_logs
### BEGIN STRIP ###
filepath = "s3://jedha-cloud-storage-lucy/youtube-/playlog_with_dates.parquet"
aws_access_key_id="."
aws_secret_access_key=".+."
hadoop_conf = spark._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3n.impl", "org.apache.hadoop.fs.s3native.NativeS3FileSystem")
hadoop_conf.set("fs.s3n.awsAccessKeyId", aws_access_key_id)
hadoop_conf.set("fs.s3n.awsSecretAccessKey", aws_secret_access_key)

In [0]:
# TODO: load the data into a PySpark DataFrame: `playlog`
# NOTE: perform the usual checks
### BEGIN STRIP ###
playlog = spark.read.load(filepath)

playlog.limit(5).toPandas()
### END STRIP ###

,user,song,datetime,year,month,dayofmonth,dayofweek,dayofyear,weekofyear
0,45058,oM9g5YZLCPI,2018-12-15 20:00:34,2018,12,15,7,349,50
1,2404,dquO_by8GI8,2018-12-15 20:02:26,2018,12,15,7,349,50
2,211,Eaay2LMdDUg,2018-12-15 20:02:37,2018,12,15,7,349,50
3,211,0aS_OjafBOs,2018-12-15 20:02:40,2018,12,15,7,349,50
4,211,dwLLQVnU8z4,2018-12-15 20:02:43,2018,12,15,7,349,50


### Aggregates

#### `firstPlay`, `lastPlay`, `playCount`, `uniquePlayCount`
For each user, we will compute these metrics:
- `firstPlay`: datetime of the first listening
- `lastPlay`: datetime of the last listening
- `playCount`: total play counts
- `uniquePlayCount`: unique play counts

We'll save all these in a new DataFrame: `users`.  
When you're done, print out the first 5 rows of `users` ordered by descending `playCount`.

In [0]:
from pyspark.sql import functions as F
import pandas as pd

In [0]:
# TODO: compute, for each user
#       - firstPlay
#       - lastPlay
#       - playCount
#       - uniquePlayCount
# Save the results in a DataFrame with name `users`
### BEGIN STRIP ###
#def compute_exprs (df):
exprs = (F.min('datetime').alias('firstPlay'),
            F.max('datetime').alias('lastPlay'),
            F.count('song').alias('playCount'),
            F.countDistinct('song').alias('uniquePlayCount'))

users = playlog.groupBy('user').agg(*exprs)
users.limit(5).toPandas()

### BEGIN STRIP ###

,user,firstPlay,lastPlay,playCount,uniquePlayCount
0,1,2014-02-14 14:18:58,2016-11-10 01:23:00,4263,3336
1,3,2014-02-14 14:19:21,2017-01-16 12:54:35,8888,688
2,5,2014-02-22 11:41:13,2017-10-18 11:48:11,106,62
3,4,2014-02-14 14:19:26,2019-03-08 17:39:48,36888,18273
4,2,2014-02-14 14:19:16,2017-12-09 19:05:27,13281,8386


In [0]:
'''IN ALTERNATIVA
def compute_aggregates(df):
  agg_exprs = (
    F.min('datetime').alias('firstPlay'),
    F.max('datetime').alias('lastPlay'),
    F.count('song').alias('playCount'),
    F.countDistinct('song').alias('uniquePlayCount')
  )
  return df.groupBy('user').agg(*agg_exprs)

users = playlog.transform(compute_aggregates)

# Alternative
# users_df = df.transform(compute_aggs('user', aggs))
OR users_df = playlog(compute_aggregates)

users.orderBy(F.desc('playCount')).limit(5).toPandas()
'''

In [0]:
users.orderBy(F.desc('playCount')).limit(5).toPandas()

,user,firstPlay,lastPlay,playCount,uniquePlayCount
0,213,2014-02-14 15:34:17,2019-04-02 06:04:08,278361,161406
1,7290,2014-04-30 20:12:41,2019-04-03 06:50:05,151424,83831
2,435,2014-02-14 19:51:09,2019-04-03 19:36:28,144598,20055
3,21950,2014-10-23 09:09:36,2019-02-06 00:54:54,126174,15075
4,6270,2014-04-13 18:45:54,2018-08-11 20:46:08,124898,9247


In [0]:
# TODO: Sanity check that all firstPlay are before than lastPlay
### BEGIN STRIP ###
users.filter(F.col('firstPlay')>F.col('lastPlay')).count()
### END STRIP ###

Out[18]: 0

In [0]:
# Another sanity check, we grouped on user, so these should be unique
# TODO: make sure all users are unique in the DataFrame
### BEGIN STRIP ##
#print(f"Total users: {users.count()}")
#print(f"Distinct users: {users.select('user').distinct().count()}")
print(f"Total_user:  {users.count()}")
print(f"Distinct_user: {users.select('user').distinct().count()}")

### END STRIP ###

Total_user: 45904
Distinct_user: 45904

### `timespan`
We will compute `timespan`: the overall span of activity from a user in days, rounded to the inferior, for example:
- if a user was active 23 hours on the service, we will say he was active 0 days
- for 53 hours, that would be 2 days of activity

We **will not** transform the `users` DataFrame in place, but instead save the result as a new DataFrame: `users_with_timespan`.

In [0]:
# TODO: Compute timespan and save the result a new DataFrame: `users_with_timespan`
### BEGIN STRIP ###
from pyspark.sql.types import IntegerType

'''MODO DEFINISCO FUNZIONE E POI UTILIZZO METHOD TRANSFORM DI PYSPARK

def compute_timespan(df):
  return df.withColumn('timespan', (
    (F.unix_timestamp('lastPlay') - F.unix_timestamp('firstPlay')) / (60**2 * 24)).cast(IntegerType()))

users_with_timespan = users.transform(compute_timespan)
users_with_timespan.limit(5).toPandas()
'''
### END STRIP ###

,user,firstPlay,lastPlay,playCount,uniquePlayCount,timespan
0,148,2014-02-14 15:53:53,2019-02-09 15:40:00,49984,14826,1820
1,463,2014-02-14 17:49:46,2018-11-18 19:18:31,39782,4601,1738
2,471,2014-02-14 21:11:44,2014-04-04 20:45:32,231,133,48
3,496,2014-02-14 22:03:35,2015-12-26 09:59:10,550,518,679
4,833,2014-02-15 16:01:16,2019-02-02 03:26:18,2601,1502,1812


In [0]:
# TODO: Compute timespan and save the result a new DataFrame: `users_with_timespan`
### BEGIN STRIP ###
from pyspark.sql.types import IntegerType

users_with_timespan = users.withColumn('timespan',((F.unix_timestamp('lastPlay') - F.unix_timestamp('firstPlay'))/(60*60*24)).cast(IntegerType()))
users_with_timespan.limit(5).toPandas()


,user,firstPlay,lastPlay,playCount,uniquePlayCount,timespan
0,148,2014-02-14 15:53:53,2019-02-09 15:40:00,49984,14826,1820
1,463,2014-02-14 17:49:46,2018-11-18 19:18:31,39782,4601,1738
2,471,2014-02-14 21:11:44,2014-04-04 20:45:32,231,133,48
3,496,2014-02-14 22:03:35,2015-12-26 09:59:10,550,518,679
4,833,2014-02-15 16:01:16,2019-02-02 03:26:18,2601,1502,1812


Let's check how this looks like, we will be using Databricks' `display` to plot an histogram of `timespan`.

In [0]:
# TODO: Plot an histogram of `timespan`
### BEGIN STRIP ###
display(users_with_timespan.select('timespan'))
### END STRIP ###

timespan
1820
1738
48
679
1812
0
493
1408
6
1577


Looking like a powerlaw, let's try to log transform.

In [0]:
# TODO: Use describe on the `timespan` column
### BEGIN STRIP ###
users_with_timespan.select('timespan').describe().toPandas().set_index('summary')
### END STRIP ###)

,summary,timespan
0,count,45904
1,mean,126.72220285813873
2,stddev,307.83969796747135
3,min,0
4,max,1874


In [0]:
# TODO: Plot a histogram of log transformed `timespan`
### BEGIN STRIP ###
display(users_with_timespan.select(F.log1p('timespan')))
### END STRIP ###

LOG1P(timespan)
7.507141079727608
7.461065514354283
3.8918202981106265
6.522092798170152
7.502738210754851
0.0
6.202535517187923
7.25063551189868
1.9459101490553132
7.363913501405819


In [0]:
# TODO: Plot a QQ-Plot of log transformed `timespan`
### BEGIN STRIP ###
display(users_with_timespan.select(F.log1p('timespan')))
### END STRIP ###

LOG1P(timespan)
7.507141079727608
7.461065514354283
3.8918202981106265
6.522092798170152
7.502738210754851
0.0
6.202535517187923
7.25063551189868
1.9459101490553132
7.363913501405819


We'll filter out users who stayed for less than a day and plot an histogram of this filtered data.

In [0]:
# TODO: Plot a histogram of log transformed `timespan` of users who stayed more than one day
### BEGIN STRIP ###
display(users_with_timespan.where(F.col('timespan') != 0).select(F.log('timespan')))
### END STRIP ###

ln(timespan)
7.506591780070841
7.460490305825338
3.8712010109078907
6.520621127558696
7.502186486602925
3.5263605246161616
6.678342114654332
7.259819610363186
5.717027701406222
7.526717561352706


### `isSingleDayUser`
What percentage of users used the service for less than one day?

What percentage of users used the service for less than one day?

In [0]:
# TODO: Compute the percentage of users who used the service for less than a day
### BEGIN STRIP ### 
users_with_timespan \
  .select(F.sum((F.col('timespan') < 1).cast(IntegerType()))) \
  .rdd.map(lambda r: r[0]).first() / users.count() * 100



Out[112]: 55.215231788079464

Wow, that's a lot! We will flag this as its own column.  
That means we will create a new Boolean column `isSingleDayUser` that is `True` if the user used the service for less than a day and `False` otherwise.

In [0]:
# TODO: Create a new column (isSingleDayUser) to flag if a user used the service for less than a day
### BEGIN STRIP ###
### BEGIN STRIP ###
users_with_single_day = users_with_timespan.withColumn('isSingleDayUser', (F.col('timespan') < 1))
users_with_single_day.limit(5).toPandas()
### END STRIP ###

,user,firstPlay,lastPlay,playCount,uniquePlayCount,timespan,isSingleDayUser
0,148,2014-02-14 15:53:53,2019-02-09 15:40:00,49984,14826,1820,False
1,463,2014-02-14 17:49:46,2018-11-18 19:18:31,39782,4601,1738,False
2,471,2014-02-14 21:11:44,2014-04-04 20:45:32,231,133,48,False
3,496,2014-02-14 22:03:35,2015-12-26 09:59:10,550,518,679,False
4,833,2014-02-15 16:01:16,2019-02-02 03:26:18,2601,1502,1812,False


### Measure of activity: `activeDaysCount` and `meanPlaycountByActiveDay`
This one is a bit harder, we want to compute:
- the number of active days for each user (not the `timespan`)
- the average play count on these active days for each user

In [0]:
# TODO: create 2 new columns
#       - activeDaysCount: the count of days each user was active
#       - dailyAvgPlayCount: the daily average playcount per user (active days only)
#       - activeDay
### BEGIN STRIP ###
def computeDailyStats(df):
  gb = df.groupBy(*(F.col(c) for c in ('user', 'year', 'dayofyear'))).count()
  exprs = (F.mean('count').alias('dailyAvgPlayCount'),
           F.count('count').alias('activeDaysCount'))
  return gb.groupBy('user').agg(*exprs)

users_with_avg = users_with_single_day.join(
  playlog.transform(computeDailyStats), 'user')
users_with_avg.limit(5).toPandas()
                                                               
### END STRIP ###

,user,firstPlay,lastPlay,playCount,uniquePlayCount,timespan,isSingleDayUser,dailyAvgPlayCount,activeDaysCount
0,148,2014-02-14 15:53:53,2019-02-09 15:40:00,49984,14826,1820,False,54.271444,921
1,463,2014-02-14 17:49:46,2018-11-18 19:18:31,39782,4601,1738,False,48.932349,813
2,471,2014-02-14 21:11:44,2014-04-04 20:45:32,231,133,48,False,23.100000,10
3,496,2014-02-14 22:03:35,2015-12-26 09:59:10,550,518,679,False,6.875000,80
4,833,2014-02-15 16:01:16,2019-02-02 03:26:18,2601,1502,1812,False,23.862385,109


In [0]:
# TODO: Plot a histogram of log of `activeDaysCount`
### BEGIN STRIP ###
display(users_with_avg.select(F.log('activeDaysCount')))
### END STRIP ###

ln(activeDaysCount)
6.825460036255307
6.70073110954781
2.302585092994046
4.382026634673881
4.6913478822291435
1.0986122886681096
1.0986122886681096
4.204692619390966
0.0
0.0


In [0]:
# TODO: Plot a histogram of log of `dailyAvgPlayCount`
### BEGIN STRIP ###
display(users_with_avg.select(F.log('dailyAvgPlayCount')))
### END STRIP ###

ln(dailyAvgPlayCount)
3.9939981969440512
3.8904387181171955
3.139832617527748
1.927891643552635
3.172303383219508
3.791736839553644
2.8134107167600364
3.401694770381686
0.6931471805599453
1.0986122886681096


## Going further
What else do you think would be interesting to compute?
What about the ratio of activity, e.g. the ratio between `timespan` and `activeDaysCount`?